In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

import os

#------------------------------------

from torch.optim import Adam, AdamW

#------------------------------------

import poseigen_seaside.basics as se
import poseigen_seaside.metrics as mex

import poseigen_compass as co

import poseigen_trident.utils as tu
import poseigen_trident.preass as pa

#------------------------------------

import R9_xps_functions as xpfus

In [2]:
data_path = "../data/R9/"
os.makedirs(data_path, exist_ok=True)

os.chdir(data_path)

os.getcwd()

'/mnt/x/Computation/Polished/case_studies/DevLoss/data/R9'

In [3]:
xpsfolder = se.NewFolder('xps')

#--------------------------------------

num_obs = [20000, 10000, 5000]
num_obs2 = num_obs[:1]

noise_lvls = [0, 0.5, 1.0] 
noise_labels = ["None", "Low", "High"]

In [4]:
snxs = se.PickleLoad('snxs')
snx2 = snxs[1]
del snx2['dim_f']

dnet = pa.DivoNet(**snx2, two_outs=False)
print(dnet)

def DivoNetReset(divonet, bn = True):
    mo = 3 if bn else 2
    divonet.P1[0*mo].reset_parameters()
    divonet.P1[1*mo].reset_parameters()
    divonet.P1[2*mo].reset_parameters()
    divonet.P1[3*mo].reset_parameters()
    return divonet

DivoNet(
  (P1): Sequential(
    (0): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1), padding=valid)
    (1): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1), padding=valid)
    (4): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv2d(10, 10, kernel_size=(1, 1), stride=(1, 1), padding=valid)
    (7): BatchNorm2d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
    (9): Conv2d(10, 1, kernel_size=(1, 1), stride=(1, 1), padding=valid)
  )
  (Comb): Identity()
  (actf): Identity()
)


In [5]:
synth_data = se.PickleLoad('synth_data')
synth_splits = se.PickleLoad('synth_splits')

In [6]:
TCR_repeats = 15

TT_base = {'loss_bind': False, 'smallest': True, 'batchsize': 512,
            'opt': Adam, 'maxepochs': 200, 'patience': 10, 'pickup': True, 
            'duds': TCR_repeats, 'mod_init_mode': [DivoNetReset, {}], 
            'statusprints': 10}

TCS_base = {'trainer': tu.TridentTrainer, 'smallest': None,
            'get_predictions': True, 'pred_rewrite': False, 
            'add_pred_args': {'batchsize': 512}, 'score_on': 2}


TCR_base = {'Splits': None, 'repeats': TCR_repeats, 
            'pickup': False, 'savemodels': True, 'returnmodel': True}

In [7]:
iterx = 50 ##########

rewrite_ensemb = False ##############

ref_ic = 0

pf_args = {'mode1': [se.RelativeChange, {'perc': True}]}

mmox = [mex.DeviaError, {'scalefactor': 1, 'expo': 2, 'root': True, 
                        'pseudo': 1e-10, 'pyt': False}]

xbe_args_base = {'repeats': TCR_repeats, 'metrics_mode': mmox,
                 'out_weights': None, 'out_bind': None,
                 'score_on': 1, 'iterx': iterx}

onesided = None
conf_alpha = 0.95

In [8]:
def XpsTables(cur_pn, cur_combs, cur_variables, mode_ref_list=[('MSE', 0)]):
    """
    Accepts a list of (mode_name, ref_ic) pairs and generates tables with a new top column level.
    """
    per_list, pdx_list, sig_mask_list = [], [], []
    per_keys = []

    for mode_name, ref_ic in mode_ref_list:
        file_ext = 'reps_bs_r2r_all_' + str(TCR_repeats) + '_' + str(ens_top) + '_' + str(iterx) + '_' +  str(ref_ic) + '_' + mode_name
        bootos = se.PickleLoad(cur_pn + file_ext)

        syrz, dprz, prpz, nlz, splz = num_synth_rep, num_dataproc_rep, len(num_obs2), len(noise_lvls), num_split_rep
        ber2rs = bootos.reshape(syrz, dprz, prpz, nlz, splz, *bootos.shape[-2:])

        ber2rs_rs1 = np.moveaxis(ber2rs, 0, -1)
        ber2rs_rs2 = np.moveaxis(ber2rs_rs1, 0, -1)
        ber2rs_rs3 = np.moveaxis(ber2rs_rs2, 2, -1)
        ber2rs_rs4 = ber2rs_rs3.reshape(*ber2rs_rs3.shape[:3], -1)

        ber2rs_mean = np.mean(ber2rs_rs4, axis=-1)
        ber2rs_se = np.std(ber2rs_rs4, axis=-1)
        ber2rs_low, ber2rs_high = co.BootstrapConfidenceInterval(ber2rs_rs4, alpha=conf_alpha, onesided=onesided, axis=-1)

        def sigo(low, high):
            if onesided == 'lesser': sigi = 0 > high
            if onesided == 'greater': sigi = low > 0
            if onesided is None: sigi = np.logical_or(0 > high, low > 0)
            return sigi

        ber2rs_sigo = sigo(ber2rs_low, ber2rs_high)

        # Wide table
        per = pd.DataFrame.from_dict({(nobs, nlabel): ber2rs_mean[inobs][inx]
                                     for inobs, nobs in enumerate(num_obs2)
                                     for inx, nlabel in enumerate(noise_labels)})
        per.index = [x[1][0] for x in cur_combs]
        per.index.names = cur_variables
        per_list.append(per)
        per_keys.append((mode_name, ref_ic))

        # Significance mask for this table (same shape as per)
        sig_mask = pd.DataFrame(
            {(nobs, nlabel): ber2rs_sigo[inobs][inx]
             for inobs, nobs in enumerate(num_obs2)
             for inx, nlabel in enumerate(noise_labels)},
            index=per.index
        )
        sig_mask_list.append(sig_mask)

        # Long table
        ber2rs_stats = np.stack([ber2rs_mean, ber2rs_se, ber2rs_low, ber2rs_high, ber2rs_sigo], axis=-1)
        ber2rs_stats_tidy = [[mode_name, ref_ic, nobs, nlabel, *c, *ber2rs_stats[inobs, inx, ic]]
                             for inobs, nobs in enumerate(num_obs2)
                             for inx, nlabel in enumerate(noise_labels)
                             for ic, c in cur_combs]
        pdx_list.append(ber2rs_stats_tidy)

    # Combine wide tables with MultiIndex columns
    per_all = pd.concat(per_list, axis=1, keys=per_keys)
    sig_mask_all = pd.concat(sig_mask_list, axis=1, keys=per_keys)

    def bold_sig(val, sig):
        return 'font-weight: bold' if sig else ''

    # Apply bolding using the significance mask
    per_all_style = per_all.style.format(precision=1)
    per_all_style = per_all_style.apply(lambda df: sig_mask_all.applymap(lambda x: 'font-weight: bold' if x else ''), axis=None)

    # Combine long tables
    tups_data = [('Mode', 'Name', ''), ('Mode', 'Ref', ''), ('Data', 'Num. Obs. ID', ''), ('Data', 'Noise ID', '')]
    tups_var = [*[('Unit', 'Subject', x) for x in cur_variables]]
    tups_stand = [('Stat', 'Mean', ''), ('Stat', 'SE', ''), ('Stat', 'Percentile', '5th'), ('Stat', 'Percentile', '95th'), ('Stat', 'Percentile', 'Sig.')]
    tups_all = [*tups_data, *tups_var, *tups_stand]

    pdx = pd.DataFrame([row for pdx_rows in pdx_list for row in pdx_rows])
    multcol = pd.MultiIndex.from_tuples(tups_all)
    pdx.columns = multcol
    pdx[('Stat', 'Percentile', 'Sig.')] = pdx[('Stat', 'Percentile', 'Sig.')].astype('bool')

    def boldo(x):
        if x[('Stat', 'Percentile', 'Sig.')] == True:
            return ['font-weight: bold'] * len(x)
        else:
            return [''] * len(x)

    pdx = pdx.sort_values([('Mode', 'Name', ''), ('Mode', 'Ref', ''), ('Data', 'Num. Obs. ID', ''), ('Data', 'Noise ID', '')])
    pdx_style = pdx.style.format(precision=4).apply(boldo, axis=1)

    return per_all, per_all_style, pdx, pdx_style

# xpLR: Learning Rate

In [22]:
num_synth_rep = 1
num_dataproc_rep = 1
num_split_rep = 1

In [23]:
pn_xpLR = se.NewFolder(xpsfolder + 'xpLR' + str(0))

xpLR_variables = ['Learning Rate']

learners = np.round(se.GeomNumSpacing(0.0001, 0.1, 10, plier = 2), 4)[::2]

xpLR_combs = [[ic, [c]] for ic, c in enumerate(learners)]
xpLR_combs

[[0, [np.float64(0.0001)]],
 [1, [np.float64(0.0007)]],
 [2, [np.float64(0.003)]],
 [3, [np.float64(0.0124)]],
 [4, [np.float64(0.05)]]]

In [24]:
ens_top = 3
ensemb_rewrite = False

In [ ]:
bo = None
actf = tu.PseudoReLU(pseudo = 0)

mod_args_base = {'batchsize': 512, 
                 'activation_f': actf, 'two_outs': False, 
                 'comb': bo, **snx2}

DZ = None

RMSE_mode = [se.AError, {'expo': 2, 'root': True}]
lmo, mmo = RMSE_mode, RMSE_mode

for syr in np.arange(num_synth_rep):
   for dpr in np.arange(num_dataproc_rep): 
      for inobs, nobs in enumerate(num_obs2):
         for inx, nlabel in enumerate(noise_labels):

            for ispl in np.arange(num_split_rep): 

               DX, DY = [synth_data[syr][dpr][inobs][nlabel][ss] 
                        for ss in ['d_x', 'd_mode']]
               
               splito = synth_splits[syr][dpr][inobs][nlabel][ispl]
               
               xpLR_TT_args = {'inps': [DX], 'out': DY, 
                              'out_std': DZ, 'out_bind': None,
                              'Split': splito,
                              'loss_mode': lmo, 'metrics_mode': mmo, 
                              **TT_base}
               
               xpLR_TCS_args = {'trainer_args': xpLR_TT_args, **TCS_base}

               for icom, com in xpLR_combs:

                  pn_t4 = os.path.join(pn_xpLR, str(syr), str(dpr), str(inobs), 
                                       str(inx), str(ispl), str(icom) + '/')
                  se.ensure_dir(pn_t4)

                  xpLR_TCR_args = {**xpLR_TCS_args, **TCR_base, 'pathname': pn_t4}

                  #-------------------------------------

                  lrz = com[0]
                  mod_args = {**mod_args_base, 'learningrate': lrz}

                  #-------------------------------------
               
                  _, _= tu.TridentCanRepeater(pa.DivoNet, mod_args, data = None, 
                                             **xpLR_TCR_args)
                  
                  #===================================================================

                  # HERE GENERATE ENSEMBLES. 

                  ps_args = {'out': DY, 'out_std': DZ, 'split': splito, 'metrics_mode': mmo}

                  pn_preds, pn_mods = [[pn_t4 + '0_' + str(ir) + s for ir in range(TCR_repeats)] 
                                       for s in ['_preds.p',  '_Mod.pt']]
                  
                  all_preds = np.array([se.PickleLoad(x) for x in pn_preds])


                  mets_all = np.array([co.PredScorer(pred, **ps_args) for pred in pn_preds])
                  stop_mets = mets_all[:, 1]

                  
                  pn_ens1 = se.NewFolder(pn_t4 + 'Boot_Ensembs')
                  pn_ens2 = se.NewFolder(pn_ens1 + str(TCR_repeats) + '_' + str(ens_top))


                  for ibe in np.arange(iterx): 

                     pn_ens3 = pn_ens2 + str(ibe) + '_Pred.p'

                     if ensemb_rewrite is False and os.path.isfile(pn_ens3): 
                           continue 

                     else: 
                           boots_idx = np.random.choice(np.arange(TCR_repeats), size = TCR_repeats, replace = True)
                           boots_mets = stop_mets[boots_idx]
                           boots_sel = np.argsort(boots_mets)[:ens_top]

                           boots_preds = np.mean(np.array([all_preds[x] for x in boots_sel]), axis = 0)
                           
                           se.PickleDump(boots_preds, pn_ens3)

In [ ]:
score_rewrite = False

MSE_mode = [se.AError, {'expo': 2, 'root': False}]
MAE_mode = [se.AError, {'expo': 1, 'root': False}]

for mode_name, mode in zip(['MSE', 'MAE'], [MSE_mode, MAE_mode]):

    for syr in np.arange(num_synth_rep):
        for dpr in np.arange(num_dataproc_rep): 
            for inobs, nobs in enumerate(num_obs2):
                for inx, nlabel in enumerate(noise_labels):

                    for ispl in np.arange(num_split_rep): 

                        DX, DY = [synth_data[syr][dpr][inobs][nlabel][ss] 
                                for ss in ['d_x', 'd_mode']]
                        
                        splito = synth_splits[syr][dpr][inobs][nlabel][ispl]
                        
                        for icom, com in xpLR_combs:

                            pn_t4 = os.path.join(pn_xpLR, str(syr), str(dpr), str(inobs), 
                                            str(inx), str(ispl), str(icom) + '/')
                            
                            se.ensure_dir(pn_t4)

                            lrz = com[0]

                            #==========================================

                            pn_ens1 = se.NewFolder(pn_t4 + 'Boot_Ensembs')
                            pn_ens2 = se.NewFolder(pn_ens1 + str(TCR_repeats) + '_' + str(ens_top))

                            pn_boots_preds = [pn_ens2 + str(ibe) + '_Pred.p' for ibe in np.arange(iterx)]

                            ps_args = {'out': DY, 'out_std': DZ, 'split': splito}

                            pn_boots_scores = pn_ens2 + 'mets_' + str(iterx) + '_' + mode_name + '.p'

                            if score_rewrite is False and os.path.isfile(pn_boots_scores): 
                                boots_scores = se.PickleLoad(pn_boots_scores)
                            
                            else: 
                                mets_all = np.array([co.PredScorer(pred, **ps_args, 
                                                                        metrics_mode = mode) 
                                                        for pred in pn_boots_preds])
                                
                                boots_scores = mets_all[:, 2]
                                
                                se.PickleDump(boots_scores, pn_boots_scores)            

                        print(f'finished: {syr, dpr, inobs, inx}')

In [26]:
mode_pairs = [('MSE', 0), ('MAE', 0)]


for mode_name, ref_ic in mode_pairs: 

    reps_bs_all, reps_bs_r2r_all = [], []

    for syr in np.arange(num_synth_rep):
        for dpr in np.arange(num_dataproc_rep): 
            for inobs, nobs in enumerate(num_obs2):
                for inx, nlabel in enumerate(noise_labels):

                    for ispl in np.arange(num_split_rep): 

                        reps_bs = []
                        
                        for icom, com in xpLR_combs:

                            pn_t4 = os.path.join(pn_xpLR, str(syr), str(dpr), str(inobs), 
                                            str(inx), str(ispl), str(icom) + '/')
                            
                            pn_ens1 = se.NewFolder(pn_t4 + 'Boot_Ensembs')
                            pn_ens2 = se.NewFolder(pn_ens1 + str(TCR_repeats) + '_' + str(ens_top))
                            pn_boots_scores = pn_ens2 + 'mets_' + str(iterx) + '_' + mode_name + '.p'
                            boots_scores = se.PickleLoad(pn_boots_scores)
                            
                            reps_bs.append(boots_scores)
                        
                        reps_bs_all.append(reps_bs) #Just get scores

                    #------------------------------------------

                    reps_comp = reps_bs[ref_ic]

                    reps_bs_r2r = [se.PairwiseFuncer(reps_bs[ic], reps_comp, **pf_args) 
                                for ic, c in xpLR_combs]

                    reps_bs_r2r_all.append(reps_bs_r2r)
                            

        reps_bs_all, reps_bs_r2r_all = (np.array(x) for x in [reps_bs_all, reps_bs_r2r_all])

        [se.PickleDump(x, pn_xpLR + y + '_' + 
                    str(TCR_repeats) + '_' + str(ens_top) + '_' + str(iterx) + 
                    '_' +  str(ref_ic) + '_' + mode_name)

        for x,y in zip([reps_bs_all, reps_bs_r2r_all],
                        ['reps_bs_all', 'reps_bs_r2r_all'])]

In [49]:
xpLR_tabs = XpsTables(pn_xpLR, xpLR_combs, xpLR_variables, mode_ref_list=mode_pairs)

xpLR_tab_main, xpLR_tab_main_sty, xpLR_tab_extra, xpLR_tab_extra_sty = xpLR_tabs

xpLR_tab_main_sty

/tmp/ipykernel_1103882/833468124.py:67: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  per_all_style = per_all_style.apply(lambda df: sig_mask_all.applymap(lambda x: 'font-weight: bold' if x else ''), axis=None)


In [50]:
xpLR_tab_extra_sty

In [51]:
xpLR_tab_main.mean(axis = 1)

Learning Rate
0.0001     0.146262
0.0007   -13.138251
0.0030   -13.870249
0.0124   -14.538517
0.0500   -13.660145
dtype: float64

# xpF: Loss Function Testing

In [9]:
optlr = 0.0124

num_synth_rep = 5
num_dataproc_rep = 3
num_split_rep = 3

ens_top = 3

xpF_rewrite = False

In [10]:
pn_xpF = se.NewFolder(xpsfolder + 'xpF' + str(2))  #################################
#T2 corrects for adding pseudo value in deviation loss instead of in the values. 

xpF_variables = ['Loss Functions']

lossos = ['MSE', 'Random_MSE',
          'MAE', 
           
          'MSLE-eps', 'MSLE-1', 'P-NLL', 'NLL', 'NMP', 

          'MDE', 'MUDE']

xpF_combs = [[ic, [c]] for ic, c in enumerate(lossos)]
xpF_combs

[[0, ['MSE']],
 [1, ['Random_MSE']],
 [2, ['MAE']],
 [3, ['MSLE-eps']],
 [4, ['MSLE-1']],
 [5, ['P-NLL']],
 [6, ['NLL']],
 [7, ['NMP']],
 [8, ['MDE']],
 [9, ['MUDE']]]

In [ ]:
bo = None
actf = tu.PseudoReLU(pseudo = 0)

mod_args = {'batchsize': 512, 
                 'activation_f': actf, 'two_outs': False, 
                 'comb': bo, **snx2, 'learningrate': optlr}

for syr in np.arange(num_synth_rep):
   for dpr in np.arange(num_dataproc_rep): 
      for inobs, nobs in enumerate(num_obs2):
         for inx, nlabel in enumerate(noise_labels):

            for ispl in np.arange(num_split_rep): 


                for icom, com in xpF_combs:
                    
                    pn_t4 = os.path.join(pn_xpF, str(syr), str(dpr), str(inobs), 
                                       str(inx), str(ispl), str(icom) + '/')
                    se.ensure_dir(pn_t4)

                    lsf = com[0]

                    DX, DY = [synth_data[syr][dpr][inobs][nlabel][ss] 
                        for ss in ['d_x', 'd_mode']]
                
                    splito = synth_splits[syr][dpr][inobs][nlabel][ispl]

                    DZ = synth_data[syr][dpr][inobs][nlabel]['d_exp'] if lsf in ['NLL', 'NMP', 'MDE', 'MUDE'] else None


                    #------------------------------------------

                    if 'Random' in lsf:
                        DY = xpfus.shuffle_train_val_y(splito, DY, has_val=True)
                        print('SHUFFLED')
                    
                    lmo, mmo = xpfus.get_loss_mode(lsf)
                    
                    #------------------------------------------

                    xpF_TT_args = {'inps': [DX], 'out': DY, 
                                'out_std': DZ, 'out_bind': None,
                                'Split': splito,
                                'loss_mode': lmo, 'metrics_mode': mmo, 
                                **TT_base}
                    
                    xpF_TCS_args = {'trainer_args': xpF_TT_args, **TCS_base}

                    xpF_TCR_args = {**xpF_TCS_args, **TCR_base, 'pathname': pn_t4}

                    #------------------------------------------

                    # THIS IS ALSO GENERATING PREDICTIONS: 
                    _, _= tu.TridentCanRepeater(pa.DivoNet, mod_args, data = None, **xpF_TCR_args)

                    #===================================================================

                    # HERE GENERATE ENSEMBLES. 

                    ps_args = {'out': DY, 'out_std': DZ, 'split': splito, 'metrics_mode': mmo}

                    pn_preds, pn_mods = [[pn_t4 + '0_' + str(ir) + s for ir in range(TCR_repeats)] 
                                        for s in ['_preds.p',  '_Mod.pt']]
                    
                    all_preds = np.array([se.PickleLoad(x) for x in pn_preds])


                    mets_all = np.array([co.PredScorer(pred, **ps_args) for pred in pn_preds])
                    stop_mets = mets_all[:, 1]

                    
                    pn_ens1 = se.NewFolder(pn_t4 + 'Boot_Ensembs')
                    pn_ens2 = se.NewFolder(pn_ens1 + str(TCR_repeats) + '_' + str(ens_top))


                    for ibe in np.arange(iterx): 

                        pn_ens3 = pn_ens2 + str(ibe) + '_Pred.p'

                        if xpF_rewrite is False and os.path.isfile(pn_ens3): 
                            continue 

                        else: 
                            boots_idx = np.random.choice(np.arange(TCR_repeats), size = TCR_repeats, replace = True)
                            boots_mets = stop_mets[boots_idx]
                            boots_sel = np.argsort(boots_mets)[:ens_top]

                            boots_preds = np.mean(np.array([all_preds[x] for x in boots_sel]), axis = 0)
                            
                            se.PickleDump(boots_preds, pn_ens3)


In [ ]:
score_rewrite = True

MSE_mode = [se.AError, {'expo': 2, 'root': False}]
MAE_mode = [se.AError, {'expo': 1, 'root': False}]

NLL_mode = [xpfus.BetaPrime_NLL, {'pyt': False, 'eps': 1e-10}]
MDE_mode = [tu.DeviaError, {'expo': 1, 'root': False, 'pseudo': 1e-10, 'pyt': False}]


for mode_name, mode in zip(['MSE', 'MAE', 'NLL', 'MDE'], 
                           [MSE_mode, MAE_mode, NLL_mode, MDE_mode]):

    for syr in np.arange(num_synth_rep):
        for dpr in np.arange(num_dataproc_rep): 
            for inobs, nobs in enumerate(num_obs2):
                for inx, nlabel in enumerate(noise_labels):

                    for ispl in np.arange(num_split_rep): 

                        DX, DY = [synth_data[syr][dpr][inobs][nlabel][ss] 
                                for ss in ['d_x', 'd_mode']]
                        
                        DY_b4noise = synth_data[syr][dpr][inobs][nlabel]['before_noise']
                        DY_real = DY_b4noise[:, [0]] / (DY_b4noise[:, [1]] + 1e-10)
                        
                        splito = synth_splits[syr][dpr][inobs][nlabel][ispl]

                        DZ = synth_data[syr][dpr][inobs][nlabel]['d_exp']
                    

                        if mode_name in ['MDE', 'NLL']: 
                            ps_args = {'out': DY, 'out_std': DZ, 'split': splito}
                        else: 
                            ps_args = {'out': DY_real, 'out_std': None, 'split': splito}
                        
                        for icom, com in xpF_combs:

                            pn_t4 = os.path.join(pn_xpF, str(syr), str(dpr), str(inobs), 
                                            str(inx), str(ispl), str(icom) + '/')
                            
                            se.ensure_dir(pn_t4)

                            #==========================================

                            pn_ens1 = se.NewFolder(pn_t4 + 'Boot_Ensembs')
                            pn_ens2 = se.NewFolder(pn_ens1 + str(TCR_repeats) + '_' + str(ens_top))

                            pn_boots_preds = [pn_ens2 + str(ibe) + '_Pred.p' for ibe in np.arange(iterx)]

                            pn_boots_scores = pn_ens2 + 'mets_' + str(iterx) + '_' + mode_name + '.p'

                            if score_rewrite is False and os.path.isfile(pn_boots_scores): 
                                boots_scores = se.PickleLoad(pn_boots_scores)
                            
                            else: 
                                mets_all = np.array([co.PredScorer(pred, **ps_args, 
                                                                        metrics_mode = mode) 
                                                        for pred in pn_boots_preds])
                                
                                boots_scores = mets_all[:, 2]
                                
                                se.PickleDump(boots_scores, pn_boots_scores)            

                        print(f'finished: {syr, dpr, inobs, inx}')

In [11]:
mode_pairs = [('MSE', 0), ('MAE', 2), ('NLL', 6), ('MDE', 8)]

In [ ]:

for mode_name, ref_ic in mode_pairs: 

    reps_bs_all, reps_bs_r2r_all = [], []

    for syr in np.arange(num_synth_rep):
        for dpr in np.arange(num_dataproc_rep): 
            for inobs, nobs in enumerate(num_obs2):
                for inx, nlabel in enumerate(noise_labels):

                    for ispl in np.arange(num_split_rep): 

                        reps_bs = []
                        
                        for icom, com in xpF_combs:

                            pn_t4 = os.path.join(pn_xpF, str(syr), str(dpr), str(inobs), 
                                            str(inx), str(ispl), str(icom) + '/')
                            
                            pn_ens1 = se.NewFolder(pn_t4 + 'Boot_Ensembs')
                            pn_ens2 = se.NewFolder(pn_ens1 + str(TCR_repeats) + '_' + str(ens_top))
                            pn_boots_scores = pn_ens2 + 'mets_' + str(iterx) + '_' + mode_name + '.p'
                            boots_scores = se.PickleLoad(pn_boots_scores)
                            
                            reps_bs.append(boots_scores)
                        
                        reps_bs_all.append(reps_bs) #Just get scores

                    #------------------------------------------

                    reps_comp = reps_bs[ref_ic]

                    reps_bs_r2r = [se.PairwiseFuncer(reps_bs[ic], reps_comp, **pf_args) 
                                for ic, c in xpF_combs]

                    reps_bs_r2r_all.append(reps_bs_r2r)
                            

        reps_bs_all, reps_bs_r2r_all = (np.array(x) for x in [reps_bs_all, reps_bs_r2r_all])

        [se.PickleDump(x, pn_xpF + y + '_' + 
                    str(TCR_repeats) + '_' + str(ens_top) + '_' + str(iterx) + 
                    '_' +  str(ref_ic) + '_' + mode_name)

        for x,y in zip([reps_bs_all, reps_bs_r2r_all],
                        ['reps_bs_all', 'reps_bs_r2r_all'])]

In [12]:
xpF_tabs = XpsTables(pn_xpF, xpF_combs, xpF_variables, mode_ref_list=mode_pairs)

xpF_tab_main, xpF_tab_main_sty, xpF_tab_extra, xpF_tab_extra_sty = xpF_tabs

xpF_tab_main_sty

/tmp/ipykernel_26095/833468124.py:67: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  per_all_style = per_all_style.apply(lambda df: sig_mask_all.applymap(lambda x: 'font-weight: bold' if x else ''), axis=None)
